<a href="https://colab.research.google.com/github/mcash8/DVBS2X-DNN-Implementation/blob/main/FFNEQ_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Train NN on DVBS2XAPSK Data


* Current FFNN has issues with loss decreasing. The purpose of this code is to take clean DVBS2X data that has only been passed through a SRRC filter and perfect channel and see if the NN can learn a perfect mapping between input and output



In [6]:
import os
import scipy.io as spio
import math
import time
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import MeanSquaredError
import matplotlib.pyplot as plt

start_time = time.time()

#parameters
num_classes = 2
batch_size = 64
epochs = 100

%cd /content/drive/My Drive/NN\ Input

#Load the data
folder = 'NoAWGN'

#import data
%cd $folder

matname = "NoAWGN.mat"
mat = spio.loadmat(matname, squeeze_me=True)
x_train = mat['x_train']
x_valid = mat['data_val']
y_train = mat['y_train']
y_valid = mat['target_val']

matname = "NoAWGN_test.mat"
mat = spio.loadmat(matname, squeeze_me=True)
x_test = mat['x_pred']

%cd ../

#cast to float32
x_train = x_train.astype('float32')
x_valid = x_valid.astype('float32')
x_test = x_test.astype('float32')

print(x_train.shape, 'train samples')
print(x_valid.shape, 'valid samples')

print(y_train.shape, 'train labels')
print(y_valid.shape, 'valid labels')

print('Label Examples:\n', y_train[0:9])
print('Input Examples:\n', x_train[0])

# Define the network
model = Sequential()
model.add(Dense(50, activation='tanh', input_dim=18))
model.add(Dense(50, activation='tanh'))
model.add(Dense(num_classes, activation='linear'))

model.summary()

model.compile(loss=keras.metrics.mean_squared_error,
            optimizer=SGD(),
            metrics=[keras.metrics.RootMeanSquaredError(name='rmse')])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=2,
                    validation_data=(x_valid, y_valid))

elapsedTime = (time.time() - start_time)/60
print("--- %.2f minutes ---" % elapsedTime)

score = model.evaluate(x_train, y_train, verbose=2)
print('Final Training MSE:', score[0])
print('Final Training RMSE:', score[1])

score = model.evaluate(x_valid, y_valid, verbose=2)
print('Final Validation MSE:', score[0])
print('Final Validation RMSE:', score[1])

#save model
savename = "NoAWGN.h5" 
model.save(savename)
!cp $savename /content/drive/MyDrive/NN\ Output/FFNNEq/Saved\ Models


#save history for graphing 
# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history.history) 
# or save to csv: 
hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

!cp $hist_csv_file /content/drive/MyDrive/NN\ Output/FFNNEq/Saved\ Models

/content/drive/My Drive/NN Input
/content/drive/My Drive/NN Input/NoAWGN
/content/drive/My Drive/NN Input
(999000, 18) train samples
(99990, 18) valid samples
(999000, 2) train labels
(99990, 2) valid labels
Label Examples:
 [[ 0.22448 -0.22448]
 [-0.22448  0.22448]
 [-0.70711  0.70711]
 [-0.22448  0.22448]
 [-0.25882  0.96593]
 [ 0.70711 -0.70711]
 [ 0.96593  0.25882]
 [ 0.25882  0.96593]
 [-0.22448 -0.22448]]
Input Examples:
 [-0.22143 -0.70589 -0.22522 -0.25788  0.7052   0.96526  0.25736 -0.2282
 -0.22296  0.22375  0.71132  0.225    0.96326 -0.70466  0.2593   0.96938
 -0.22735 -0.2217 ]
